## El método de disparo - The Shooting Method
### Métodos Numéricos 2022

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

#### Método de tiro
Este es un método por el cual ciertos sistemas de ecuaciones ordinarias se pueden resolver como problemas de frontera, es decir, dando valores iniciales para algunas variables y *valores finales * para otras. 
Un caso donde se puede hacer esto es cuando resolvemos problemas que provienen de ecuaciones de segundo grado, las cuales en la clasificación de ecuaciones en derivadas parciales serían denominadas *elípticas*.

Este es el caso cuando tenemos un problema para las ecuacioens de Newton:

\begin{equation}
\frac{d^2\vec{y}}{dt^2} = f(t,\vec{y},\frac{d\vec{y}}{dt}) 
\end{equation}

Donde queremos proponer el siguiente problema:

*Dadas dos posiciones $x_i$ y $x_f$, y un intervalo temporal $(t_i,t_f)$, encuentre una velocidad $dx/dt(t_i)$ que $x(t_i)=x_i$
y $x(t_f)= x_f$.* 

Veamos un caso simple que se puede tratar analíticamente.
\begin{equation}
\frac{d{x^2}}{dt^2} = p(t)\frac{d{x}}{dt}+q(t)x+ r(t) 
\end{equation}

Suponiendo las condiciones donde se puede demostrar que la solución existe: sean *p(t)*, *q(t)* y *r(t)* contínuos en el intervalo $t \in [t_i, t_f]$ que supondremos acotado y *q(t)*> 0.

Luego sabemos que este problema tiene dos soluciones linealmente independietes, *u(t)* y *v(t)*, para nuestro análisis es conveniente elegir que sean tales que $u(t_i)= x_i$, $du/dt(t_i)= 0$ y $v(t_i)=0$, $dv/dt(t_i)= 1$.




Luego, como el sistema es lineal, la suma de estas soluciones y la multiplicación de las mismas por constantes son también soluciones, siendo la solución general una combinación lineal de ellas de la forma:

\begin{equation}
     x(t) = A_u u(t) + A_v v(t) 
\end{equation}

Luego la solución que estamos buscando está dada por las constantes

\begin{equation}
                       A_u = 1 , A_v = \frac{x_f-u(t_f)}{v(t_f)}
\end{equation}                      
  
 El hecho de que $v(t_f) \neq 0 $ sigue de la condición de que *q(t)* sea postiva.  
  
  
\begin{equation}
\frac{d{x}}{dt}  = A_v =  \frac{x_f-u(t_f)}{v(t_f)}
\end{equation}


using Plots 
"""
    RK4(f,y0,t0,h,p...)

Hace un paso del método del metodo clásico de RK de orden 4: 
    f  : función que nos da la tangente como en (y,t)
    y0 : y inicial
    t0 : t inicial
    h  : dt
    p  : extra parameters

# Examples
```julia-repl
julia> 
function f(y,t)
    return -y + sin(2π*t)
end
h= 0.1
RK4(f,1,0,h)
0.9342307485981525
```
"""


In [3]:
def funcRK4(f,y0,t0,h):
    np.k1 = h*f(y0,t0)
    np.k2 = h*f(y0 + k1/2,t0 + h/2)
    np.k3 = h*f(y0 + k2/2,t0 + h/2)
    np.k4 = h*f(y0 + k3,t0 + h)
    return y0 + (k1 + 2*k2 + 2*k3 + np.k4)/6

def funcRK4p(f,y0,t0,h,p):
    np.k1 = h*f(y0,t0,p)
    np.k2 = h*f(y0 + k1/2,t0 + h/2,p)
    np.k3 = h*f(y0 + k2/2,t0 + h/2,p)
    np.k4 = h*f(y0 + k3,t0 + h,p)
    return y0 + (k1 + 2*k2 + 2*k3 + np.k4)/6



""" def funcODEmultidim(Method,f,y0,(a,b),N):
    h = (b-a)/(N-1)
    t = zeros(N)
    w = zeros((length(y0),N))
    t[1] = a
    w[:,1] = y0
    for i in 2:N
        t[i] = t[i-1] + h
        w[:,i] = Method(f,w[:,i-1],t[i-1],h)         

    return (t[:],w[:,:])

def funcODEmultidimp(Method,f,y0,(a,b),N,p):
    h = (b-a)/(N-1)
    t = zeros(N)
    w = zeros((length(y0),N))
    t[1] = a
    w[:,1] = y0
    for i in 2:N
        t[i] = t[i-1] + h
        w[:,i] = Method(f,w[:,i-1],t[i-1],h,p)         
    
    return (t[:],w[:,:]) 
    """

In [5]:
def funcODEmultidim(Method,f,y0,a,b,N):
    h = (b-a)/(N-1)
    t = zeros(N)
    w = zeros((length(y0),N))
    t[1] = a
    w[:,1] = y0
    for i in 2:N
    t[i] = t[i-1] + h
    w[:,i] = Method(f,w[:,i-1],t[i-1],h)         
   
    return (t[:],w[:,:])

def funcODEmultidimp(Method,f,y0,a,b,N,p):
    h = (b-a)/(N-1)
    t = zeros(N)
    w = zeros((length(y0),N))
    t[1] = a
    w[:,1] = y0
    for i in 2:N
    t[i] = t[i-1] + h
    w[:,i] = Method(f,w[:,i-1],t[i-1],h,p)         
   
    return (t[:],w[:,:])


### Ejemplo 

Veamos un ejemplo:

Elegimos $p(t) = -1$, $q(t) = 1$, $r(t) = sin(2\pi t)$. Luego el sistema queda de la forma:

$$ 
\frac{dy^1}{dt} = y^2, \;\;\;\;\;\;\; \frac{dy^2}{dt} = - y^2 + y^1 + sin(2 \pi  t)
$$

Lo queremos resolver en el intervalo $(0,4)$ y con condiciones de contorno $x_i = 0$, $x_f = 2$.

In [6]:
intervalo = (0.0,4.0)
N = 100

def f(y,t):
    return [y[2], -y[2] + y[1] + sin(2*pi*t)]


f0= np.array([2,3])
print(f0)


[2 3]


In [7]:
# primera solución 
x_i = 0.0
y0=np.array([x_i, 0.0])

funcODEmultidim(funcRK4,f,x_i,0.0,4.0,N)

NameError: name 'zeros' is not defined

In [87]:
# segunda solución 

y0=np.array([0.0, 1.0])

funcODEmultidim(funcRK4,f,y0,intervalo,N)

TypeError: funcODEmultidim() missing 1 required positional argument: 'N'

In [46]:
p = plot(title="soluciones")
plot!(p,t[:],u[1,:], label="u")
#plot!(p,t,u[2,:], label="udot")
plot!(p,t[:],v[1,:], label="v")
#plot!(p,t,v[2,:], label="vdot")
p

SyntaxError: invalid syntax (<ipython-input-46-a2f15bb212bb>, line 2)

In [ ]:
x_f=2
Aᵥ = (x_f - u[1,end])/v[1,end]

plot(t,u[1,:] .+ Aᵥ*v[1,:], legend=:topleft, label="solución completa")

### Tarea:

Vea que puede tomar valores negativos de $q(t)$ y todavía obtener soluciones, trate de ver que sucede en esos casos probando distintas formas para $q(t)$ negativas y sin la fuente. Ayuda: trate de hacer que la solución $v$ se haga cero en $t_f$. 

### Caso general: 

El caso lineal considerado anterirmente es simple pues al poder sumar soluciones la respuesta es simple. Que sucede en el caso no-lineal general. Pues que tendremos que emplear métodos más sofisticados para obtener las soluciones. Iremos directamente en busca del valor de la derivada primera en $t_i$ que hace que la condición de contorno en $t_f$ se satisfaga.

Es decir, encontrando una solución con condiciones iniciales $(x(t_i) = x_i, \frac{dx}{dt}|_{t_i} = v)$, plantearemos la función:

$$
F(v) := x(t_f) - x_f,
$$
y buscaremos un cero para la misma, por alguno de los métodos que nos permiten hacer esto, por ejemplo bisección, secante, regula-falsi o Newton.

A continuación veremos los de bisección y Newton.


### Bisección:

Tendremos que iniciar el algoritmo encontrando dos valores de $v$ tales que $F(v_1)*F(v_2) < 0$ pero luego 
el algoritmo nos conducirá lentamente a la solución. Notemos que calcular $F(v)$ para estos problemas es caro en término de cómputos, para cada valor de $v$ debemos hacer una integración numérica.

En este caso usaremos como ejemplo:  

3. $1\leq t\leq 2$, comience con $h=0.05$
$$
\ddot{x}\,=\,\frac{(t\,\dot{x} )^2\,-9 x^2+4 t^6}{t^5},\;\;\;x(1)=0\;,\;\;x(2)=\ln{(256)} \,.
$$
Solución exacta $x=t^3\,\ln{(t})$.

In [88]:
def g(y,t)
    return [y[2]; ((t*y[2])**2 - (3*y[1])**2 + 4*t**6)/t**5]:


def F(v,p)
    t_i, t_f, x_i, x_f, N, f = p
    intervalo = (t_i, t_f)
    y0=[x_i; v]
    (t,u) = ODEmultidim(RK4,f,y0,intervalo,N);
    return u[1,end] - x_f


SyntaxError: invalid syntax (<ipython-input-88-f784220cb2c3>, line 1)

In [67]:
def biseccion(f,a,b,epsx=1e-5,epsf=1e-5,n=100,p):
    fa = f(a,p)
    fx = f(b,p)
    if fa*fx > 0; error("no hay raíz en [a,b]") end
    for i in 1:n
        x = 0.5*(a+b)
        fx = f(x,p)
        if fa*fx >= 0
            a = x  # La raiz esta en [x,b], i.e. la mitad derecha de [a,b]
            fa = fx
        else
            b = x  # La raiz esta en [a,x), i.e. la mitad izquierda de [a,b].
        end        
        R = abs(b-a)/abs(b+a)
        F = abs(fx)
        if R < epsx && F < epsf
            return x,fx,R,F,i
        end
    end
    error("convergencia fallida")


SyntaxError: invalid syntax (<ipython-input-67-01e8f81f3195>, line 4)

In [69]:
t_i = 1
t_f = 2
x_i = 0.0
x_f = log(256)
N = 1000
p =  t_i, t_f, x_i, x_f, N, g

F(1.0,p)

NameError: name 'log' is not defined

In [61]:
F(2.0,p)

NameError: name 'F' is not defined

In [62]:
a = 1.0
b = 2.0
biseccion(F,a,b;epsx=1e-5,epsf=1e-5,n=100,p)

SyntaxError: invalid syntax (<ipython-input-62-07af8cc9d3a6>, line 3)

In [63]:
y0 = [0.0, 1.0]
intervalo = (1.0,2.0)
N = 1000
(t,u) = ODEmultidim(RK4,g,y0,intervalo,N);

NameError: name 'ODEmultidim' is not defined

In [64]:
y(t) = t^3*log(t)
scatter(t[1:50:end],u[1,1:50:end],ms=4,label="aproximada", legend=:topleft)
plot!(t,y.(t), label="exacta")

SyntaxError: invalid syntax (<ipython-input-64-342d124eda52>, line 2)

### Método de Newton-Raphson

Ahora veremos como hacer este problema de forma más eficiente. Note que por bisección debimos hacer 19 iteraciones, o sea un total de 20 integraciones de nuestra ecuación.

Recordemos que la iteración de Newton-Raphson para encontrar una raíz de un función diferenciable $f(x)$ está dada por,

$$ 
x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)} \;\;\;\;\;\;\; f'= \frac{df}{dx}
$$

Por lo tanto debemos encontrar la derivada primera de la función $F(v) = x(t_f) - x_f$.

En esta función la dependencia de $v$ viene implícita al calcular $x(t_f)$ con condición inicial 
$(x(t_i)=x_i, \frac{dx}{dt}|_{t_i} = v)$. Es decir debemos ver como depende $x(t_f)$ con respecto al dato inicial. Para ello pensemos en calcular una solución $y(t)$ con dato inicial $(x(t_i)=x_i, \frac{dx}{dt}|_{t_i} = v)$ y otra, $y_p(t)$ con dato inicial $(x(t_i)=x_i, \frac{dx}{dt}|_{t_i} = v + \delta v)$. Luego, la derivada vendrá dada por,

$$
F'(v) = \frac{x_p(t_f) - x(t_f)}{\delta v}
$$

Y por lo tanto necesitamos ver cómo varía esta diferencia a primer orden.
Pero notemos que $\delta y(t) := y_p(t) - y(t)$ satisface a primer orden la siguiente ecuación:

$$
\frac{d\delta y(t)}{dt} = f(y_p(t),t) - f(y(t)) \approx \frac{\partial f}{\partial y} \; \delta y 
$$
Es decir, para encontrar la derivada de $F$ necesitamos resolver una ecuación lineal en $\delta y$ con condiciones iniciales, $(x(t_i)=0, \frac{dx}{dt}|_{t_i} = \delta v)$, como es una ecuación lineal, y luego debemos dividir por $\delta v$, simplemente dividimos las condiciones iniciales por $\delta v$ y resolvemos este problema alternativo con condiciones iniciales, $(x(t_i)=0, \frac{dx}{dt}|_{t_i} = 1)$.

Veamos ahora en detalle como es este sistema. Notemos que la ecuación es un sistema, o sea que $y(t)$ tiene dos índices, $y(t) = (y^1(t),y^2(t))$ lo mismo sucede con $f$, por lo tanto lo que debemos calcular es el Jacobiano de la transformación:

$$
J^i{}_j = \frac{\partial f^i}{\partial y^j} 
        = \left( 
            \begin{array}{cc}
            \frac{\partial f^1}{\partial y^1} & \frac{\partial f^1}{\partial y^2} \\
            \frac{\partial f^2}{\partial y^1} & \frac{\partial f^2}{\partial y^2}
            \end{array}
        \right)
$$

Quedando el sistema linealizado (notemos que estas derivadas dependen de la solución del problema, $y(t)$), 

$$
\frac{d}{dt}
    \left(
    \begin{array}{c}
    y^1 \\
    y^2
    \end{array}
    \right)
        = \left( 
            \begin{array}{cc}
            \frac{\partial f^1}{\partial y^1} & \frac{\partial f^1}{\partial y^2} \\
            \frac{\partial f^2}{\partial y^1} & \frac{\partial f^2}{\partial y^2}
            \end{array}
        \right)
        \left(
    \begin{array}{c}
    y^1 \\
    y^2
    \end{array}
    \right)
$$

En la práctica este sistema lineal se adiciona al sistema no-lineal y se resuelve como un sistema aumentado.

### Ejemplo: 

Veamos el sistema anterior pero con el método de Newton-Raphson. Recordando que,

$$
f(y) = (f^1, f^2) = (y^2, \frac{(ty^2)^2 - (3y^2)^2 + 4t^6}{t^5})
$$   

Tenemos que,
$$
\left( 
            \begin{array}{cc}
            \frac{\partial f^1}{\partial y^1} & \frac{\partial f^1}{\partial y^2} \\
            \frac{\partial f^2}{\partial y^1} & \frac{\partial f^2}{\partial y^2}
            \end{array}
\right) 
= 
\left( 
            \begin{array}{cc}
            0  & 1 \\
            \frac{-18y^1}{t^5} & \frac{2y^2}{t^3}
            \end{array}
\right) 
$$
Resultando en el siguiente sistema ampliado: 

$$ 
\frac{d}{dt}
    \left(
    \begin{array}{c}
    y^1 \\
    y^2 \\
    y^3 \\
    y^4 \\
    \end{array}
    \right)
    = 
    \left(
    \begin{array}{c}
    y^2 \\
    (y^2, \frac{(ty^2)^2 - (3y^2)^2 + 4t^6}{t^5}) \\
    y^4 \\
    \frac{-6y^1}{t^5} * y^3 + \frac{2y^2}{t^3} * y^4 \\
    \end{array}
    \right)
    \;\;\;\;\;\;\;\; y^3 = \delta y^1, \;\; y^4 = \delta y^2
$$
